In [ ]:
from bsf_env import init_spark, init_mariadb_engine,set_spark_verbosity
from bsf_dbutilities import DBUtils
from pyspark.sql import functions as F
import pandas as pd

try:
    spark.stop()
except NameError:
    # SparkSession doesn't exist
    pass

spark = init_spark("bsf_utilities", log_level="WARN", show_progress=False, enable_ui=True,process_option='default')
engine = init_mariadb_engine()
ingest_ts = spark.sql("SELECT current_timestamp()").collect()[0][0]

# ─── Setup Database Communications ──────────────────────────────────────
db = DBUtils(spark, ingest_ts)
db.spark_stats()

pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 200)         # Expand width to avoid line breaks


from delta.tables import DeltaTable
import datetime, os
from pyspark.sql import functions as F

# List all databases
databases = [db.name for db in spark.catalog.listDatabases()]

for database in databases:
    print(f"\n=== Database: {database} ===\n")
    
    # Get all tables in this database
    tables = spark.catalog.listTables(database)
    if not tables:
        print("No tables found.")
        continue

    table_info = []
    
    for t in tables:
        full_table_name = f"{database}.{t.name}"
        row_count = spark.table(full_table_name).count()
        table_type = t.tableType
    
        # Default: no max date
        from pyspark.sql import functions as F
        import datetime
        import os
        
        max_date = None
        try:
            df_table = spark.table(full_table_name)
            
            if "LastLoadDate" in df_table.columns:
                max_date = df_table.agg(F.max("LastLoadDate")).collect()[0][0]
                field = "LastLoadDate"
            elif "StockDate" in df_table.columns:
                max_date = df_table.agg(F.max("StockDate")).collect()[0][0]
                field = "StockDate"
            elif "ChangeDate" in df_table.columns:
                max_date = df_table.agg(F.max("ChangeDate")).collect()[0][0]
                field = "ChangeDate"
            else:
                try:
                    dt = DeltaTable.forName(spark, full_table_name)
                    history = dt.history(1)
                    max_date = history.select(F.max("timestamp")).collect()[0][0]
                    field = "timestamp"
                except:
                    max_date = None
        
            # --- Normalize to datetime ---
            if isinstance(max_date, datetime.date) and not isinstance(max_date, datetime.datetime):
                max_date = datetime.datetime.combine(max_date, datetime.time.min)
        
        except:
            # Non-Delta fallback
            table_location = (
                spark.sql(f"DESCRIBE FORMATTED {full_table_name}")
                     .filter("col_name='Location'")
                     .select("data_type")
                     .collect()[0][0]
            )
            max_date = datetime.datetime.fromtimestamp(os.path.getmtime(table_location))


    
        table_info.append((t.name, row_count, field, max_date, table_type))
    
    df_table_info = spark.createDataFrame(
        table_info,
        schema=["TableName", "RowCount","DateField", "MaxDate", "TableType"]
    )
    
    df_table_info.show(truncate=False)
    # 🚨 Safety stop — prevents accidental full execution
    raise RuntimeError("⚠️ This notebook is blocked. Do NOT run all cells without checking!")

In [4]:
spark.stop()

In [3]:
sdf = spark.table("bsf.companyfundamental           ")
pdf = sdf.limit(10).toPandas()
pdf

,CompanyId,FundamentalDate,PeRatio,PegRatio,PbRatio,ReturnOnEquity,NetProfitMarginTTM,TotalDebtToEquity,CurrentRatio,InterestCoverage,EpsChangeYear,RevChangeYear,Beta,ShortIntToFloat
0,34592,2024-10-06,-0.82861,-0.00171,0.74147,-75.0444,-2683.44,0.0497,1.40801,0.0,0.0,501.288,1.39637,0.0
1,34592,2024-10-20,-0.92649,-0.00169,0.74147,-75.0444,-2683.44,0.0497,1.40801,0.0,0.0,501.288,1.39637,0.0
2,34592,2024-10-27,-0.90227,-0.00167,0.74147,-75.0444,-2683.44,0.0497,1.40801,0.0,0.0,501.288,1.39637,0.0
3,34592,2024-11-03,-0.94647,-0.00179,0.74147,-75.0444,-2683.44,0.0497,1.40801,0.0,0.0,501.288,1.39637,0.0
4,34592,2024-11-17,-0.79348,-0.00158,0.74147,-75.0444,-2683.44,0.0497,1.40801,0.0,0.0,501.288,1.34436,0.0
5,34592,2024-11-24,-0.70847,-0.00136,0.74147,-75.0444,-2683.44,0.0497,1.40801,0.0,0.0,501.288,1.34436,0.0
6,34592,2024-12-01,-0.68976,-0.00131,0.74147,-75.0444,-2683.44,0.0497,1.40801,0.0,0.0,501.288,1.34436,0.0
7,34592,2024-12-08,-0.66687,-0.00136,0.74147,-75.0444,-2683.44,0.0497,1.40801,0.0,0.0,501.288,1.37500,0.0
8,34592,2024-12-15,-0.69354,-0.00135,0.74147,-75.0444,-2683.44,0.0497,1.40801,0.0,0.0,501.288,1.37500,0.0
9,34592,2024-12-22,-0.65236,-0.00133,0.74147,-75.0444,-2683.44,0.0497,1.40801,0.0,0.0,501.288,1.37500,0.0


In [4]:
sdf = spark.table("bsf.companystockhistory_watermark")
pdf = sdf.limit(10).toPandas()
pdf

#pdf.head(10).T

,CompanyId,LastLoadedDate
0,30524,2025-09-22
1,31025,2025-09-22
2,31247,2025-09-22
3,34497,2025-09-22
4,67725,2025-09-22
5,84699,2025-09-22
6,29531,2025-09-22
7,30357,2025-09-22
8,31004,2025-09-22
9,33550,2025-09-22


In [5]:
sdf = spark.table("bsf.history_signals_last")
pdf = sdf.limit(10).toPandas()
pdf

,CompanyId,StockDate,Open,High,Low,Close,TomorrowClose,Return,TomorrowReturn,MA,MA_slope,UpTrend_MA,DownTrend_MA,MomentumUp,MomentumDown,ConfirmedUpTrend,ConfirmedDownTrend,Volatility,LowVolatility,HighVolatility,SignalStrength,SignalStrengthHybrid,ActionConfidence,BullishStrengthHybrid,BearishStrengthHybrid,SignalDuration,PatternAction,CandleAction,UpTrend_Return,CandidateAction,Action,TomorrowAction,TimeFrame
0,52,2025-09-22,0.0510,0.05500,0.0350,0.03500,NaN,-0.338999,NaN,0.053590,0.092603,True,False,False,True,False,False,1.530954e-01,False,True,4,1.465623,1.465623,0.798956,1.465623,152.0,Buy,Sell,False,Sell,Sell,Hold,Swing
1,145,2025-09-22,0.0190,0.01900,0.0190,0.01900,NaN,0.000000,NaN,0.019000,-0.045226,False,True,False,True,False,True,1.430176e-02,False,False,0,0.892754,0.000000,0.892754,0.892754,86.0,Sell,Hold,False,Buy,Hold,Hold,Swing
2,641,2025-09-22,0.0135,0.01350,0.0135,0.01350,NaN,0.000000,NaN,0.012000,0.287554,True,False,True,False,True,False,2.539276e-01,False,True,0,0.709860,0.000000,0.709860,0.709860,193.0,Buy,Hold,True,Buy,Hold,Hold,Swing
3,972,2025-09-22,0.0290,0.02900,0.0290,0.02900,NaN,0.000000,NaN,0.029000,0.000000,False,False,False,False,False,False,1.000000e-08,False,False,0,0.000000,0.000000,0.000000,0.000000,61.0,Hold,Hold,False,Hold,Hold,Hold,Swing
4,1394,2025-09-22,0.0031,0.00335,0.0031,0.00335,NaN,-0.042857,NaN,0.003176,-0.009357,False,True,True,False,False,False,6.496555e-02,True,False,1,0.399176,0.399176,0.399176,0.399176,175.0,Sell,Hold,True,Sell,Sell,Hold,Swing
5,1493,2025-09-22,0.0013,0.00130,0.0013,0.00130,NaN,0.000000,NaN,0.001300,0.048387,True,False,False,False,False,False,2.635231e-02,True,False,2,0.634120,0.000000,0.634120,0.634120,127.0,Buy,Hold,False,Buy,Hold,Hold,Swing
6,1849,2025-09-22,0.0410,0.04300,0.0410,0.04300,NaN,0.000000,NaN,0.042418,0.020890,True,False,True,False,True,False,5.315796e-02,True,False,0,0.481975,0.000000,0.481975,0.481975,174.0,Buy,Hold,True,Buy,Hold,Hold,Swing
7,1851,2025-09-22,0.0099,0.00990,0.0099,0.00990,NaN,0.000000,NaN,0.007380,-0.102190,False,True,True,False,False,False,2.814933e-01,False,True,2,0.678016,0.344683,0.678016,0.344683,130.0,Sell,Buy,True,Sell,Sell,Hold,Swing
8,2217,2025-09-22,0.1000,0.10950,0.0900,0.09660,NaN,-0.086093,NaN,0.109460,0.081065,True,False,False,True,False,False,1.769237e-01,True,False,1,0.378258,0.378258,0.378258,0.378258,159.0,Buy,Hold,False,Buy,Buy,Hold,Swing
9,2285,2025-09-22,0.0214,0.02700,0.0202,0.02700,NaN,0.273585,NaN,0.023380,0.035430,True,False,True,False,True,False,2.480436e-01,True,False,1,0.414802,0.414802,0.414802,0.414802,163.0,Buy,Hold,True,Buy,Buy,Hold,Swing


In [6]:
sdf = spark.table("bsf.history_signals")
pdf = sdf.limit(10).toPandas()
pdf

,CompanyId,StockDate,Open,High,Low,Close,TomorrowClose,Return,TomorrowReturn,MA,MA_slope,UpTrend_MA,DownTrend_MA,MomentumUp,MomentumDown,ConfirmedUpTrend,ConfirmedDownTrend,Volatility,LowVolatility,HighVolatility,SignalStrength,SignalStrengthHybrid,ActionConfidence,BullishStrengthHybrid,BearishStrengthHybrid,SignalDuration,PatternAction,CandleAction,UpTrend_Return,CandidateAction,Action,TomorrowAction,TimeFrame
0,397,2024-08-08,0.1200,0.1390,0.11700,0.1350,0.13500,0.000000,0.000000,NaN,NaN,False,False,False,False,False,False,NaN,False,False,0,NaN,0.000000,NaN,NaN,1.0,Hold,Hold,False,Hold,Hold,Hold,Short
1,397,2024-08-09,0.1255,0.1390,0.11595,0.1350,0.15500,0.000000,0.148148,0.13500,NaN,False,False,False,False,False,False,1.000000e-08,False,False,0,0.000000,0.000000,0.000000,0.000000,1.0,Hold,Hold,False,Hold,Hold,Buy,Short
2,397,2024-08-12,0.1200,0.1550,0.11100,0.1550,0.14390,0.148148,-0.071613,0.14500,0.074074,True,False,True,False,True,False,8.553337e-02,False,False,1,0.372621,0.372621,0.372621,0.372621,2.0,Buy,Hold,True,Buy,Buy,Hold,Short
3,397,2024-08-13,0.1503,0.1590,0.13490,0.1439,0.16060,-0.071613,0.116053,0.14945,0.030690,True,False,True,False,False,False,9.239746e-02,False,True,0,0.536517,0.000000,0.536517,0.536517,3.0,Buy,Hold,False,Buy,Hold,Hold,Short
4,397,2024-08-14,0.1430,0.1610,0.12550,0.1606,0.15000,0.116053,-0.066002,0.15225,0.018735,True,False,True,False,True,False,9.100344e-02,False,True,0,0.793906,0.000000,0.793906,0.793906,3.0,Buy,Hold,True,Buy,Hold,Sell,Short
5,397,2024-08-15,0.1580,0.1610,0.13690,0.1500,0.15100,-0.066002,0.006667,0.15530,0.020033,True,False,True,False,False,False,9.190236e-02,False,True,5,1.712806,1.712806,0.712806,1.712806,4.0,Buy,Sell,False,Sell,Sell,Buy,Short
6,397,2024-08-16,0.1590,0.1590,0.14200,0.1510,0.14000,0.006667,-0.072848,0.15050,-0.030908,False,True,False,True,False,False,8.407211e-02,False,True,3,1.282255,1.282255,1.282255,0.282255,5.0,Buy,Buy,True,Buy,Buy,Sell,Short
7,397,2024-08-19,0.1420,0.1549,0.14000,0.1400,0.13900,-0.072848,-0.007143,0.14550,-0.033223,False,True,False,True,False,True,8.434284e-02,True,False,1,0.354552,0.354552,0.354552,0.354552,6.0,Sell,Hold,False,Sell,Sell,Hold,Short
8,397,2024-08-20,0.1400,0.1490,0.13000,0.1390,0.13100,-0.007143,-0.057554,0.13950,-0.041237,False,True,False,True,False,True,7.904739e-02,True,False,0,0.732311,0.000000,0.732311,0.732311,7.0,Sell,Hold,False,Sell,Hold,Hold,Short
9,397,2024-08-21,0.1348,0.1390,0.13000,0.1310,0.14245,-0.057554,0.087405,0.13500,-0.032258,False,True,False,True,False,True,7.718213e-02,True,False,0,0.873558,0.000000,0.873558,0.873558,7.0,Sell,Hold,False,Sell,Hold,Buy,Short


In [7]:
sdf = spark.table("bsf.final_candidates_enriched")
pdf = sdf.limit(10).toPandas()
pdf

,RunTimestamp,CompanyId,TimeFrame,StockDate,Open,High,Low,Close,TomorrowClose,Return,TomorrowReturn,MA,MA_slope,UpTrend_MA,DownTrend_MA,MomentumUp,MomentumDown,ConfirmedUpTrend,ConfirmedDownTrend,Volatility,LowVolatility,HighVolatility,SignalStrength,SignalStrengthHybrid,ActionConfidence,BullishStrengthHybrid,BearishStrengthHybrid,SignalDuration,PatternAction,CandleAction,UpTrend_Return,CandidateAction,Action,TomorrowAction,BuyRank,BestModel,BestModel_RMSE,BestModel_MAE,BestModel_MAPE,BestModel_DirectionAcc,BestModel_R2,BestModel_AdjR2,Pred_Sklearn,PredictedReturn_Sklearn,Pred_Linear,PredictedReturn_Linear,Linear_RMSE,Linear_MAE,Linear_MAPE,Linear_DirectionAcc,Linear_R2,Linear_AdjR2,Pred_Lasso,PredictedReturn_Lasso,Lasso_RMSE,Lasso_MAE,Lasso_MAPE,Lasso_DirectionAcc,Lasso_R2,Lasso_AdjR2,Pred_Ridge,PredictedReturn_Ridge,Ridge_RMSE,Ridge_MAE,Ridge_MAPE,Ridge_DirectionAcc,Ridge_R2,Ridge_AdjR2,Pred_XGBoost,PredictedReturn_XGBoost,XGBoost_RMSE,XGBoost_MAE,XGBoost_MAPE,XGBoost_DirectionAcc,XGBoost_R2,XGBoost_AdjR2,MaxPredictedReturn,Phase2_Rank,SARIMAX_PredictedClose,SARIMAX_PredictedReturn,WeightedScore,AIC,MlType,TradingSymbol,Name
0,20250923_203153,521,Swing,2025-09-22,0.03160,0.03300,0.02910,0.03180,NaN,0.025806,NaN,0.031060,-0.228515,False,True,False,True,False,True,0.084115,False,True,3,1.897305,1.897305,1.897305,0.897305,179.0,Sell,Buy,False,Buy,Buy,Hold,2,XGBoost,0.002646,0.001874,0.022224,0.832740,0.998986,0.998908,0.033636,0.057730,0.035007,0.100836,0.007950,0.003844,0.048445,0.832740,0.990844,0.990142,-0.001631,-1.051276,0.054253,0.033251,0.300125,0.355872,0.573527,0.540847,0.020454,-0.356793,0.015163,0.008718,0.082466,0.701068,0.966689,0.964136,0.037199,0.169790,0.002646,0.001874,0.022224,0.832740,0.998986,0.998908,0.169790,3,0.031744,-0.001760,0.101170,-1582.097906,automl,AOTVF,ASCOT RES LTD
1,20250923_203153,897,Swing,2025-09-22,0.02400,0.02400,0.02400,0.02400,NaN,0.600000,NaN,0.019360,-0.059829,False,True,True,False,False,False,0.262148,False,True,2,1.148691,1.148691,1.148691,0.648691,152.0,Sell,Hold,True,Buy,Buy,Hold,8,XGBoost,0.000930,0.000634,0.024795,0.730627,0.989682,0.988816,0.024206,0.008573,0.023502,-0.020731,0.002684,0.001270,0.045230,0.405904,0.914081,0.906864,0.023033,-0.040281,0.008599,0.006273,0.261347,0.394834,0.118232,0.044164,0.025548,0.064491,0.006506,0.004831,0.190352,0.343173,0.495311,0.452917,0.024384,0.016004,0.000930,0.000634,0.024795,0.730627,0.989682,0.988816,0.064491,8,0.021908,-0.087164,0.003829,-2024.370951,automl,AUMMF,AuMega Metals Ltd
2,20250923_203153,1581,Short,2025-09-22,0.03300,0.03560,0.03300,0.03508,NaN,0.073768,NaN,0.033875,0.036884,True,False,True,False,True,False,0.126072,True,False,1,0.813563,0.813563,0.813563,0.813563,156.0,Buy,Hold,True,Buy,Buy,Hold,30,Linear,0.000572,0.000365,0.009823,0.779359,0.994084,0.993607,0.035216,0.003867,0.035119,0.001124,0.000572,0.000365,0.009823,0.779359,0.994084,0.993607,0.036604,0.043456,0.007195,0.006136,0.168771,0.249110,0.063102,-0.012571,0.036572,0.042537,0.006195,0.005133,0.140331,0.569395,0.305311,0.249202,0.035081,0.000018,0.000549,0.000399,0.010601,0.768683,0.994547,0.994107,0.043456,14,0.034159,-0.026261,0.015569,-2533.054957,automl,AWSL,ATLANTIC PWR & INFRA
3,20250923_203153,1643,Long,2025-09-22,0.00450,0.00490,0.00450,0.00490,NaN,0.088889,NaN,0.004655,-0.024518,False,True,True,False,False,False,0.039255,True,False,3,1.623478,1.623478,1.623478,0.623478,176.0,Sell,Buy,True,Buy,Buy,Hold,1,XGBoost,0.000208,0.000139,0.033980,0.765125,0.989715,0.988884,0.004831,-0.013986,0.004802,-0.020075,0.000306,0.000181,0.045438,0.743772,0.977714,0.975914,0.004835,-0.013364,0.002048,0.001696,0.460252,0.149466,0.000000,-0.080769,0.004667,-0.047479,0.001806,0.001554,0.416482,0.576512,0.221922,0.159077,0.004870,-0.006132,0.000208,0.000139,0.033980,0.765125,0.989715,0.988884,-0.006132,12,0.004790,-0.022527,-0.012690,-3344.849524,automl,AXIM,AXIM BIOTECHNOLOGIES
4,20250923_203153,1643,Swing,2025-09-22,0.00450,0.00490,0.00450,0.00490,NaN,0.088889,NaN,0.004614,-

In [8]:
sdf = spark.table("bsf.final_candidates")
pdf = sdf.limit(10).toPandas()
pdf

,CompanyId,TimeFrame,StockDate,Open,High,Low,Close,TomorrowClose,Return,TomorrowReturn,MA,MA_slope,UpTrend_MA,DownTrend_MA,MomentumUp,MomentumDown,ConfirmedUpTrend,ConfirmedDownTrend,Volatility,LowVolatility,HighVolatility,SignalStrength,SignalStrengthHybrid,ActionConfidence,BullishStrengthHybrid,BearishStrengthHybrid,SignalDuration,PatternAction,CandleAction,UpTrend_Return,CandidateAction,Action,TomorrowAction,BuyRank,BestModel,BestModel_RMSE,BestModel_MAE,BestModel_MAPE,BestModel_DirectionAcc,BestModel_R2,BestModel_AdjR2,Pred_Sklearn,PredictedReturn_Sklearn,Pred_Linear,PredictedReturn_Linear,Linear_RMSE,Linear_MAE,Linear_MAPE,Linear_DirectionAcc,Linear_R2,Linear_AdjR2,Pred_Lasso,PredictedReturn_Lasso,Lasso_RMSE,Lasso_MAE,Lasso_MAPE,Lasso_DirectionAcc,Lasso_R2,Lasso_AdjR2,Pred_Ridge,PredictedReturn_Ridge,Ridge_RMSE,Ridge_MAE,Ridge_MAPE,Ridge_DirectionAcc,Ridge_R2,Ridge_AdjR2,Pred_XGBoost,PredictedReturn_XGBoost,XGBoost_RMSE,XGBoost_MAE,XGBoost_MAPE,XGBoost_DirectionAcc,XGBoost_R2,XGBoost_AdjR2,MaxPredictedReturn,Phase2_Rank,SARIMAX_PredictedClose,SARIMAX_PredictedReturn,WeightedScore,AIC,MlType,TradingSymbol,Name,Phase3_Rank
0,100167,Daily,2025-09-22,0.0200,0.03000,0.0200,0.02000,NaN,65.666667,NaN,0.010043,0.165837,True,False,True,False,True,False,20.801939,False,True,4,1.999948,0.999948,0.999948,1.999948,41.0,Buy,Hold,True,Buy,Buy,Hold,14,XGBoost,0.000222,0.000117,23.965739,0.924051,0.998176,0.997998,0.059578,1.978905,0.600873,29.043656,0.002445,0.001216,246.314138,0.620253,0.779743,0.758329,0.003931,-0.803441,0.004842,0.004620,1267.855456,0.645570,0.136029,0.052032,0.010645,-0.467759,0.004055,0.003307,923.894960,0.611814,0.393814,0.334879,0.015556,-0.222219,0.000222,0.000117,23.965739,0.924051,0.998176,0.997998,29.043656,1,0.016226,-0.188692,17.350717,-2072.654464,automl,WRCDF,"WIRECARD AG, BERLIN",1
1,92779,Daily,2025-09-22,0.0115,0.01265,0.0115,0.01265,NaN,0.193396,NaN,0.012204,-0.021196,False,True,False,True,False,False,0.207573,True,False,2,1.002751,1.002751,1.002751,0.502751,166.0,Sell,Buy,True,Buy,Buy,Hold,28,XGBoost,0.000830,0.000615,0.049286,0.753571,0.997741,0.997548,0.014707,0.162609,0.013489,0.066329,0.002362,0.001615,0.136591,0.532143,0.981719,0.980160,0.032810,1.593639,0.016537,0.013329,1.174648,0.392857,0.103985,0.027581,0.037355,1.952980,0.011549,0.008866,0.751232,0.428571,0.562974,0.525708,0.012598,-0.004140,0.000830,0.000615,0.049286,0.753571,0.997741,0.997548,1.952980,3,0.012650,0.000000,1.171788,-2287.052933,automl,STGZ,STARGAZE ENTMT GROUP,2
2,59260,Daily,2025-09-22,0.0015,0.00165,0.0015,0.00160,NaN,0.032258,NaN,0.001693,-0.024691,False,True,False,True,False,False,0.050199,True,False,2,1.067693,1.067693,1.067693,0.567693,182.0,Sell,Hold,True,Buy,Buy,Hold,22,Linear,0.000130,0.000083,0.023138,0.818505,0.992884,0.992280,0.001955,0.221996,0.001665,0.040569,0.000130,0.000083,0.023138,0.818505,0.992884,0.992280,0.003036,0.897631,0.001319,0.001031,0.264696,0.362989,0.263236,0.200653,0.002549,0.592897,0.001153,0.000893,0.218106,0.626335,0.437481,0.389699,0.002107,0.316706,0.000168,0.000109,0.030664,0.800712,0.988075,0.987062,0.897631,2,0.001600,0.000000,0.538579,-3603.627631,automl,IGPK,INTEGRATED CANNABIS,3
3,67984,Daily,2025-09-22,0.0539,0.05670,0.0539,0.05670,NaN,0.051948,NaN,0.058197,-0.009964,False,True,False,True,False,False,0.080576,False,True,4,1.491882,1.491882,1.491882,0.491882,165.0,Sell,Buy,True,Buy,Buy,Hold,6,XGBoost,0.002772,0.001885,0.024518,0.857143,0.997127,0.996861,0.057045,0.006088,0.059701,0.052933,0.005245,0.002749,0.032772,0.764479,0.989717,0.988762,0.083729,0.476702,0.051672,0.035789,0.629236,0.478764,0.001945,-0.090701,0.064654,0.140284,0.014377,0.009303,0.125533,0.621622,0.922731,0.915559,0.052742,-0.069798,0.002772,0.001885,0.024518,0.857143,0.997127,0.996861,0.476702,7,0.056432,-0.004718,0.284134,-1555.210275,automl,MATEF,BLOCKMATE VENTURES I,4
4,98577,Daily,2025-09-22,0.0027,0.00270,0.0027,0.00270,NaN,0.038462,NaN,0.002614,0.005495,True,False,True,False,True,False,0.012163,True,False,

In [9]:
sdf = spark.table("bsf.final_candidates_enriched")

# Group by CompanyId, TimeFrame, StockDate
counts_df = (
    sdf.groupBy("TimeFrame", "StockDate")
      .agg(F.count("*").alias("row_count"))
      .orderBy("TimeFrame", "StockDate")
)

counts_df.show(50, truncate=False)


+---------+----------+---------+
|TimeFrame|StockDate |row_count|
+---------+----------+---------+
|Daily    |2025-09-22|15       |
|Long     |2025-09-22|15       |
|Short    |2025-08-29|1        |
|Short    |2025-09-22|14       |
|Swing    |2025-09-22|15       |
+---------+----------+---------+



In [11]:
# Assume you have a Spark DataFrame
sdf = spark.table("bsf.companystockhistory")

# 1️⃣ Show the schema (data types included)
sdf.printSchema()

# 2️⃣ Get a list of column names
# print(sdf.columns)

# 3️⃣ Show first few rows with all columns (default shows truncated view)
sdf.show(truncate=False)  

# 4️⃣ For more detailed metadata about columns
sdf.dtypes  # Returns a list of (column_name, data_type)


root
 |-- CompanyId: long (nullable = true)
 |-- StockDate: date (nullable = true)
 |-- OpenPrice: double (nullable = true)
 |-- HighPrice: double (nullable = true)
 |-- LowPrice: double (nullable = true)
 |-- ClosePrice: double (nullable = true)
 |-- StockVolume: double (nullable = true)

['CompanyId', 'StockDate', 'OpenPrice', 'HighPrice', 'LowPrice', 'ClosePrice', 'StockVolume']
+---------+----------+---------+---------+--------+----------+-----------+
|CompanyId|StockDate |OpenPrice|HighPrice|LowPrice|ClosePrice|StockVolume|
+---------+----------+---------+---------+--------+----------+-----------+
|64490    |2024-08-28|0.054    |0.054    |0.0535  |0.0535    |3200.0     |
|64490    |2024-08-29|0.0533   |0.0533   |0.0533  |0.0533    |2499.0     |
|64490    |2024-08-30|0.053    |0.053    |0.053   |0.053     |1334.0     |
|64490    |2024-09-03|0.041    |0.053    |0.041   |0.0416    |5275.0     |
|64490    |2024-09-04|0.0416   |0.0416   |0.0416  |0.0416    |0.0        |
|64490    |2024

[('CompanyId', 'bigint'),
 ('StockDate', 'date'),
 ('OpenPrice', 'double'),
 ('HighPrice', 'double'),
 ('LowPrice', 'double'),
 ('ClosePrice', 'double'),
 ('StockVolume', 'double')]

In [10]:
# 🚨 Safety stop — prevents accidental full execution
raise RuntimeError("⚠️ This notebook is blocked. Do NOT run all cells without checking!")


RuntimeError: ⚠️ This notebook is blocked. Do NOT run all cells without checking!